# Start

In [1]:
from insightface.app import FaceAnalysis
app = FaceAnalysis(name='buffalo_l')
app.prepare(ctx_id=-1, det_size=(640,640))  # -1 = CPU, 0 = first GPU
print("InsightFace ready.")

Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: C:\Users\Parvez/.insightface\models\buffalo_l\1k3d68.onnx landmark_3d_68 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: C:\Users\Parvez/.insightface\models\buffalo_l\2d106det.onnx landmark_2d_106 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: C:\Users\Parvez/.insightface\models\buffalo_l\det_10g.onnx detection [1, 3, '?', '?'] 127.5 128.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: C:\Users\Parvez/.insightface\models\buffalo_l\genderage.onnx genderage ['None', 3, 96, 96] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: C:\Users\Parvez/.insightface\models\buffalo_l\w600k_r50.onnx recognition ['None', 3, 112, 112]

In [5]:
import torch 

print("Number of GPU: ", torch.cuda.device_count())
print("GPU Name: ", torch.cuda.get_device_name())

Number of GPU:  0


AssertionError: Torch not compiled with CUDA enabled

In [3]:
# enroll_face.py
import json, cv2, numpy as np
from pathlib import Path
from insightface.app import FaceAnalysis

def enroll(out_dir="whitelist", cam=0, shots=20, device="cpu"):
    Path(out_dir).mkdir(parents=True, exist_ok=True)
    app = FaceAnalysis(name="buffalo_l")
    ctx_id = -1 if device=="cpu" else 0
    app.prepare(ctx_id=ctx_id, det_size=(640,640))

    cap = cv2.VideoCapture(cam)
    embs = []
    print("[Enroll] Look at the camera; press q to stop early.")
    while len(embs) < shots:
        ok, frame = cap.read()
        if not ok: break
        faces = app.get(frame)
        if faces:
            f = max(faces, key=lambda x: (x.bbox[2]-x.bbox[0])*(x.bbox[3]-x.bbox[1]))
            embs.append(f.normed_embedding.astype(float))
            # feedback box
            x1,y1,x2,y2 = map(int, f.bbox)
            cv2.rectangle(frame,(x1,y1),(x2,y2),(0,255,0),2)
        cv2.imshow("Enroll", frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    cap.release(); cv2.destroyAllWindows()

    if not embs:
        print("[Enroll] No face captured."); return
    mean_emb = np.mean(np.stack(embs, axis=0), axis=0)
    out = Path(out_dir) / "creator_embedding.json"
    out.write_text(json.dumps({"embedding": mean_emb.tolist()}), encoding="utf-8")
    print(f"[Enroll] Saved: {out}")

if __name__ == "__main__":
    enroll()


Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: C:\Users\Parvez/.insightface\models\buffalo_l\1k3d68.onnx landmark_3d_68 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: C:\Users\Parvez/.insightface\models\buffalo_l\2d106det.onnx landmark_2d_106 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: C:\Users\Parvez/.insightface\models\buffalo_l\det_10g.onnx detection [1, 3, '?', '?'] 127.5 128.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: C:\Users\Parvez/.insightface\models\buffalo_l\genderage.onnx genderage ['None', 3, 96, 96] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: C:\Users\Parvez/.insightface\models\buffalo_l\w600k_r50.onnx recognition ['None', 3, 112, 112]

In [2]:
import numpy as np
def cosine_distance(a, b):
    a = a / (np.linalg.norm(a)+1e-9)
    b = b / (np.linalg.norm(b)+1e-9)
    return 1 - float(np.dot(a, b))


# Blur the faces